In [10]:
import pandas as pd
import numpy as np

from ta import momentum, trend, volume
from binance import Client
from key import PUB_KEY, SEC_KEY

# Get Historical Data

In [11]:
# Connecting to Binance
client = Client(PUB_KEY, SEC_KEY)
# Get historical data
df = pd.DataFrame(client.get_historical_klines('BTCUSDT',
                             Client.KLINE_INTERVAL_1MINUTE,
                                      str(3000) + ' minutes ago UTC'))

# First 6 column represent Time and OHLCV values
df = df.iloc[:, :6]
df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']

# Convert first column into datetime
df['Time'] = pd.to_datetime(df['Time'], unit='ms')
for column in df.columns:
    if column != 'Time':
        df[column] = df[column].astype(float)

# Set Time as index
df.set_index('Time', inplace=True)

# Generate Technical Indicators

In [12]:
df

,Open,High,Low,Close,Volume
Time,,,,,
2024-07-15 17:38:00,63339.24,63339.24,63304.51,63330.00,24.94415
2024-07-15 17:39:00,63329.99,63329.99,63311.56,63316.01,12.41827
2024-07-15 17:40:00,63316.01,63353.50,63306.00,63350.01,18.71572
2024-07-15 17:41:00,63350.00,63350.01,63310.35,63312.48,16.95423
2024-07-15 17:42:00,63312.49,63322.00,63270.18,63311.57,23.75641
...,...,...,...,...,...
2024-07-17 19:33:00,64737.91,64737.92,64650.00,64696.53,21.00891
2024-07-17 19:34:00,64696.53,64715.00,64668.05,64668.05,9.55257
2024-07-17 19:35:00,64668.06,64668.06,64631.55,64656.27,7.94489


In [13]:
df['SMA50'] = df['Close'].rolling(50).mean()
df['SMA200'] = df['Close'].rolling(200).mean()
df['RSI'] = momentum.rsi(df['Close'])
df['MACD'] = trend.macd(df['Close'])
df['MACD_sig'] = trend.macd_signal(df['Close'])
df['MACD_diff'] = trend.macd_diff(df['Close'])
df['Momentum'] = (df['Close'] - df['Close'].shift(30))/df['Close'].shift(30)
df['ADX'] = trend.adx(df['High'], df['Low'], df['Close'])
df['-DI'] = trend.adx_neg(df['High'], df['Low'], df['Close'])
df['+DI'] = trend.adx_pos(df['High'], df['Low'], df['Close'])


# Generate Trading Signals

For each TAs, a buy, hold and sell signal will be generated. They will bear the weight of 1, 0 and -1 respectively

In [14]:
df.dropna(inplace=True)
df

,Open,High,Low,Close,Volume,SMA50,SMA200,RSI,MACD,MACD_sig,MACD_diff,Momentum,ADX,-DI,+DI
Time,,,,,,,,,,,,,,,
2024-07-15 20:57:00,63746.01,63750.00,63728.67,63737.80,17.83774,63550.8986,63452.91450,59.258067,47.405909,48.381783,-0.975874,0.002703,32.183392,13.342501,30.308558
2024-07-15 20:58:00,63737.80,63777.00,63728.67,63769.37,16.26397,63560.2860,63455.11135,62.046047,48.715209,48.448468,0.266741,0.002502,33.053419,12.386392,32.139969
2024-07-15 20:59:00,63769.36,63772.66,63728.68,63731.36,12.06209,63569.8800,63457.18810,56.989535,46.153718,47.989518,-1.835800,0.002002,33.861302,11.573632,30.031034
2024-07-15 21:00:00,63731.36,63764.06,63702.86,63704.00,25.24944,63577.5900,63458.95805,53.603195,41.438320,46.679278,-5.240959,0.002262,33.676480,14.314650,27.342379
2024-07-15 21:01:00,63704.00,63764.06,63704.00,63761.71,12.31667,63587.3842,63461.20420,59.120877,41.875334,45.718490,-3.843155,0.003174,33.504859,13.077277,24.978876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-17 19:33:00,64737.91,64737.92,64650.00,64696.53,21.00891,64549.2314,64350.77975,61.357557,53.891967,45.810277,8.081690,0.002168,27.305973,24.334491,29.615157
2024-07-17 19:34:00,64696.53,64715.00,64668.05,64668.05,9.55257,64551.7122,64352.15985,56.504665,49.109083,46.470038,2.639045,0.001736,26.054699,22.458232,27.331743
2024-07-17 19:35:00,64668.06,64668.06,64631.55,64656.27,7.94489,64555.3578,64353.71120,54.581707,43.862447,45.948520,-2.086073,0.002045,24.394521,27.159765,25.673972


In [15]:
#SMA Golden Cross & Death Cross
sma_buy_cond = (df['SMA50'] > df['SMA200']) & (df['SMA50'].shift(1) < df['SMA200'])
sma_sell_cond = (df['SMA50'] < df['SMA200']) & (df['SMA50'].shift(1) > df['SMA200'])
df['Signal_sma'] = np.where(sma_buy_cond, 1, np.where(sma_sell_cond, -1, 0))

In [16]:
# MACD Below 0 Cross
macd_buy_cond = (df['MACD_diff'] > 0) & (df['MACD_diff'].shift(1) < 0) & (df['MACD'] < 0) &(df['MACD_sig'] < 0)
macd_sell_cond = (df['MACD_diff'] < 0) & (df['MACD_diff'].shift(1) > 0) & (df['MACD'] > 0) &(df['MACD_sig'] > 0)
df['Signal_macd'] = np.where(macd_buy_cond, 1, np.where(macd_sell_cond, -1, 0))

In [17]:
# RSI Overbought & Oversold
rsi_buy_cond = (df['RSI'] > 30) & (df['RSI'].shift(1) < 30)
rsi_sell_cond = (df['RSI'] < 70) & (df['RSI'].shift(1) > 70)
df['Signal_rsi'] = np.where(rsi_buy_cond, 1, np.where(rsi_sell_cond, -1, 0))

In [18]:
# ADX Trend Confirmation
adx_buy_cond = (df['ADX'] > 25) & (df['+DI'] > df['-DI'])
adx_sell_cond = (df['ADX'] > 25) & (df['-DI'] > df['+DI'])
df['Signal_adx'] = np.where(adx_buy_cond, 1, np.where(adx_sell_cond, -1, 0))

In [19]:
df

,Open,High,Low,Close,Volume,SMA50,SMA200,RSI,MACD,MACD_sig,MACD_diff,Momentum,ADX,-DI,+DI,Signal_sma,Signal_macd,Signal_rsi,Signal_adx
Time,,,,,,,,,,,,,,,,,,,
2024-07-15 20:57:00,63746.01,63750.00,63728.67,63737.80,17.83774,63550.8986,63452.91450,59.258067,47.405909,48.381783,-0.975874,0.002703,32.183392,13.342501,30.308558,0,0,0,1
2024-07-15 20:58:00,63737.80,63777.00,63728.67,63769.37,16.26397,63560.2860,63455.11135,62.046047,48.715209,48.448468,0.266741,0.002502,33.053419,12.386392,32.139969,0,0,0,1
2024-07-15 20:59:00,63769.36,63772.66,63728.68,63731.36,12.06209,63569.8800,63457.18810,56.989535,46.153718,47.989518,-1.835800,0.002002,33.861302,11.573632,30.031034,0,-1,0,1
2024-07-15 21:00:00,63731.36,63764.06,63702.86,63704.00,25.24944,63577.5900,63458.95805,53.603195,41.438320,46.679278,-5.240959,0.002262,33.676480,14.314650,27.342379,0,0,0,1
2024-07-15 21:01:00,63704.00,63764.06,63704.00,63761.71,12.31667,63587.3842,63461.20420,59.120877,41.875334,45.718490,-3.843155,0.003174,33.504859,13.077277,24.978876,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-17 19:33:00,64737.91,64737.92,64650.00,64696.53,21.00891,64549.2314,64350.77975,61.357557,53.891967,45.810277,8.081690,0.002168,27.305973,24.334491,29.615157,0,0,0,1
2024-07-17 19:34:00,64696.53,64715.00,64668.05,64668.05,9.55257,64551.7122,64352.15985,56.504665,49.109083,46.470038,2.639045,0.001736,26.054699,22.458232,27.331743,0,0,0,1
2024-07-17 19:35:00,64668.06,64668.06,64631.55,64656.27,7.94489,64555.3578,64353.71120,54.581707,43.862447,45.948520,-2.086073,0.002045,24.394521,27.159765,25.673972,0,-1,0,0
